# setup paperspace fast-ai based VM for torch

## Installs and Imports

Since a lot of code for object detection is same and has to be rewritten by everyone, torchvision contributers have provided us with helper codes for training, evaluation and transformations.

Let's clone the repo and copy the libraries into working directory

In [1]:
# Download TorchVision repo to use some files from
# references/detection


# !git clone https://github.com/pytorch/vision.git
# !git checkout v0.3.0

# !cp vision/references/detection/utils.py ./
# !cp vision/references/detection/transforms.py ./
# !cp vision/references/detection/coco_eval.py ./
# !cp vision/references/detection/engine.py ./
# !cp vision/references/detection/coco_utils.py ./

In [2]:
%%bash
apt update
DEBIAN_FRONTEND=noninteractive apt-get install -y python3-opencv
#tz prompt. dpkg fix...
pip install opencv-python
#pip install cv2

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://ppa.launchpad.net/git-core/ppa/ubuntu bionic InRelease [20.8 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2830 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2297 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [22.8 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [982 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1517 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [29.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3293 kB]
Get:13 http://ar



debconf: delaying package configuration, since apt-utils is not installed


In [3]:
!pip install pycocotools 
# --quiet

     |████████████████████████████████| 106 kB 17.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=273779 sha256=7cb96933bc4b07e77dcd1cc108c6fb58837b673e8429ca0ec05f6d3c2d20cbf3
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools


In [4]:
# !pip install "torch>=1.09,<1.10"

In [5]:
# !pip install "torch==1.10.0"

In [6]:
pip list

Package                 Version
----------------------- ---------
absl-py                 1.0.0
albumentations          1.1.0
argcomplete             1.12.3
argon2-cffi             21.1.0
attrs                   21.2.0
backcall                0.2.0
beautifulsoup4          4.10.0
bleach                  4.1.0
blis                    0.7.5
boto3                   1.20.4
botocore                1.23.4
bravado                 11.0.3
bravado-core            5.17.0
brotlipy                0.7.0
cachetools              4.2.4
captum                  0.4.1
catalogue               2.0.6
catalyst                21.10
certifi                 2021.10.8
cffi                    1.14.6
chardet                 4.0.0
click                   8.0.3
conda                   4.10.3
conda-build             3.21.5
conda-package-handling  1.7.3
configparser            5.1.0
cryptography            3.4.8
cycler                  0.11.0
cymem                   2.0.6
debugpy                 1.5.1
decorator         

In [7]:
conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             4.5                       1_gnu  
absl-py                   1.0.0                    pypi_0    pypi
albumentations            1.1.0                    pypi_0    pypi
argcomplete               1.12.3                   pypi_0    pypi
argon2-cffi               21.1.0                   pypi_0    pypi
attrs                     21.2.0                   pypi_0    pypi
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.10.0             pyh06a4308_0  
blas                      1.0                         mkl  
bleach                    4.1.0                    pypi_0    pypi
blis                      0.7.5                    pypi_0    pypi
boto3                     1.20.4                   pypi_0    pypi
botocore                  1.23.4                   pypi_0    p


Note: you may need to restart the kernel to use updated packages.


Lets import the libraries

In [8]:
# Basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# xml library for parsing xml files
from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image


In [9]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

## Dataset 

build the images dataset.

In [10]:
!pwd

/notebooks/ml635e2/ml635j


# Fin.

That's it for the notebook. 
